<a href="https://colab.research.google.com/github/emil2397/useful_tools/blob/main/vk_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VK parser

Данный скрипт обрабатывает ссылки и печатает посты в сообщество.


In [ ]:
! pip install vk_api

     |████████████████████████████████| 48 kB 2.0 MB/s 


In [1]:
import json
import requests
import datetime
import vk_api
import pandas as pd


user_id_1 = str(##)
user_id_2 = str(##)


group_id=str(211075646)


login_1 = "+7###"
password_1 = "##"

login_2 = "+7##"
password_2 = "##"


vk_session_1 = vk_api.VkApi(login_1, password_1)
vk_session_2 = vk_api.VkApi(login_2, password_2)

try:
    vk_session_1.auth(token_only=True)
    vk_session_2.auth(token_only=True)

except vk_api.AuthError as error_msg:
    print(error_msg)

# Scrapping

user_terminator_1 = vk_api.VkTools(vk_session_1)
user_terminator_2 = vk_api.VkTools(vk_session_2)


user_token_1="###"
user_token_2="###"


def scrap_vk_wall(link, reference_date, key_words, user_terminator):
  
  owner_id=link.split('/')[-1].replace('id','').replace('club','')

  wall = user_terminator.get_all('wall.get', 5, {'owner_id': f"{owner_id}"})

  posts_grabed = {}

  for post in wall['items'][1:]:

    
    post_date = datetime.datetime.fromtimestamp(post['date'])

    if  post_date.strftime('%Y-%m-%d') >= reference_date.strftime("%Y-%m-%d"):
      
  
      if any([key_word in post["text"].lower() for key_word in key_words]):

        posts_grabed[post_date] = {}
        posts_grabed[post_date]['post_link'] = f"{link}?w=wall{owner_id}_{post['id']}"
        posts_grabed[post_date]["text"]=post['text']

        if len(post['attachments'])>1:
          posts_grabed[post_date]["photo"]=[]
          for photo in post['attachments']:
            if 'photo' in photo.keys():
              posts_grabed[post_date]["photo"].append(photo['photo']['sizes'][-1]['url'])
        else:
          posts_grabed[post_date]["photo"]=[post['attachments'][0]['photo']['sizes'][-1]['url']]

  return posts_grabed





## Фото загрузка


album_id_1=str(##)
album_id_2=str(##)

def publish_pic(url, user_id, album_id, user_token):

  params = (
      ('owner_id', user_id), # user
      ('album_id', album_id),  # user
      ('access_token', user_token), 
      ('v','5.81')        
  )

  response = requests.get('https://api.vk.com/method/photos.getUploadServer', params=params)

  # print(response.text)


  upload_server = json.loads(response.text)['response']['upload_url']
  # print(upload_server)

  response = requests.get(url)

  file = open("sample_image.png", "wb")
  file.write(response.content)
  file.close()

  files = {'file1': open('sample_image.png','rb')}

  response = requests.post(upload_server, files=files)

  # print(response.text)

  img_hash = json.loads(response.text)['hash']
  photos_list = json.loads(response.text)['photos_list']
  server = json.loads(response.text)['server']

  params = (
      ('owner_id', user_id),
      ('album_id', album_id),  
      ('hash', img_hash),
      ('photos_list', photos_list),    
      ('server', server),
      ('caption', 'test image'),    
      ('access_token', user_token), 
      ('v','5.81')    
  )

  response = requests.get('https://api.vk.com/method/photos.save', params=params)
  # print(response.text)

  photo_id = json.loads(response.text)["response"][0]["id"]

  return photo_id

def publish_pic_bull(post, user_id, album_id, user_token):
  
  photo_ids = []
  
  for url in post["photo"]:
    photo_id = publish_pic(url, user_id, album_id, user_token)
    photo_ids.append(photo_id)
  
  return photo_ids

## печать поста

def publish_post(post, user_id,user_token):

  params = (
      ('owner_id', f'-{group_id}'),
      ('from_group', '1'),  
      ('message', post['text']),
      ('attachments', ','.join([f"photo{user_id}_{x}" for x in post["photo"]])),
      ('access_token', user_token), 
      ('v','5.81')    
  )

  response = requests.get('https://api.vk.com/method/wall.post', params=params)

  # print(response.text)

  print("post published!")


In [2]:
## Работа пайплайна 

links=pd.read_csv('vk_list.csv')

links_ = links['href_list'].values
key_words=["распродаж","новинк","налич"]

start_time = datetime.datetime.now()

yest = start_time - datetime.timedelta(days=45)

import time 

log = ""
scrap_ok=0


In [5]:
import random 

terminators = [     
    {
    "user_id": user_id_1,
    "user_terminator": user_terminator_1,
    "user_token": user_token_1,
    "album_id": album_id_1
    },

    {
    "user_id": user_id_2,
    "user_terminator": user_terminator_2,
    "user_token": user_token_2,
    "album_id": album_id_2
    },

]

# switcher between terminators
i=0
#seconds
sleep_between_posts = 2
sleep_between_links=2

active_terminator = terminators[1]

for link in links_[1:]:

  ind = list(links_).index(link)

  if ind % 5==0:
    i+=1
    i = i % len(terminators)
    active_terminator = terminators[i]
    print(i)

  posts_grabed={}
  print(f"work with {link}")
  log+=f"work with {link} \n"

  try:
    posts_grabed = scrap_vk_wall(link, yest, key_words, active_terminator["user_terminator"])
    
    if len(posts_grabed)>=1:
      scrap_ok+=1

      for key in posts_grabed.keys():
        post_ = posts_grabed[key]
        photo_ids=publish_pic_bull(post_,active_terminator["user_id"], 
                                   active_terminator["album_id"], 
                                   active_terminator["user_token"])
        post_["photo"]=photo_ids
        publish_post(post_, active_terminator["user_id"], 
                     active_terminator["user_token"])
        time.sleep(sleep_between_posts)

      print(f"{link} success")
      log+=f"{link} success \n"
    
  except Exception as e: print(e)

  print(f"number of ok scraps: {scrap_ok}")
  time.sleep(sleep_between_links)

end_time = datetime.datetime.now()

text_file = open("log.txt", "w")
text_file.write(log)
text_file.close()

print(f"Done! I started at {start_time} and finished at {end_time}")


    


work with https://vk.com/id511446223
post published!


KeyboardInterrupt: ignored